# COGS 108 - Final Project 

# Overview

*Fill in your overview here*

# Names

- Karen Rodriguez Pinto
- Enlin Wei
- Yumi Minami 
- Kanami Hannah Tanaka

# Group Members IDs

- A15161856
- A12938737
- A12669124
- A########

# Research Question

*Fill in your research question here*

## Background and Prior Work

*Fill in your background and prior work here* 

References (include links):
- 1)
- 2)

# Hypothesis


*Fill in your hypotheses here*

# Dataset(s)

*Fill in your dataset information here*

(Copy this information for each dataset)
- Dataset Name:
- Link to the dataset:
- Number of observations:

1-2 sentences describing each dataset. 

If you plan to use multiple datasets, add 1-2 sentences about how you plan to combine these datasets.

# Setup

In [1]:
## YOUR CODE HERE
import random as rd
import pandas as pd

# Data Cleaning

Describe your data cleaning steps here.

In [2]:
## YOUR CODE HERE
## FEEL FREE TO ADD MULTIPLE CELLS PER SECTION

In [3]:
""""
Gets a certain number of lines at random from a string.
--INPUTS--
    input_string: string from which to extract a random number of lines
    num_lines_to_get: number of lines to get at random
    already_got: list of lines that are already picked, to prevent picking same line twice
--OUTPUTS--
    output: output string containing 'num_lines_to_get' number of random lines
"""
def get_rand_lines(input_string,num_lines_to_get,already_got=[]):
    # output to save random lines
    output = ""
    # max lines in input string
    MAX_LINES = len(input_string)
    # number of lines that have been picked
    ith_time = 0
    # while we haven't gotten desired number of lines
    while ith_time < num_lines_to_get:
        # picks random line number in range of 0 to MAX_LINES-1
        ith_line = rd.randint(0, MAX_LINES-1)
        # checks if random line number has already been picked
        if ith_line not in already_got:
            # adds randomly picked line to output
            output = output + input_string[ith_line]
            # saves that line number to already_got
            already_got.append(ith_line)
            # increments ith_time
            ith_time = ith_time + 1
    return output

In [4]:
# opens csv file and reads lines in it into list of strings
cit_lines = open("parking-citations.csv", 'r').readlines()
# uses get_rand_lines method to extract 10000 lines from it
# aleady_got=[0] because we have to get the header
cit_lines_10000_rand = get_rand_lines(cit_lines, 10000, already_got=[0])
# writes in the headers (line 0 in 'cit_lines'), and the 10000 random lines
outfile = open("10000-rand-parking-citations.csv", 'w').write(cit_lines[0] + cit_lines_10000_rand)

In [5]:
# reads csv into df
cit_df = pd.read_csv("10000-rand-parking-citations.csv")
print(cit_df)

      Ticket number           Issue Date  Issue time Meter Id  Marked Time  \
0        4269770352  2015-10-18T00:00:00       423.0      NaN          NaN   
1        4309183535  2017-06-08T00:00:00      1137.0      NaN          NaN   
2        4267825343  2015-10-19T00:00:00      1846.0    BF242          NaN   
3        1115829540  2016-10-26T00:00:00       735.0      NaN          NaN   
4        4329728756  2018-04-04T00:00:00       448.0      NaN          NaN   
5        4274197550  2016-01-13T00:00:00      1243.0    CB502          NaN   
6        4280681856  2016-04-11T00:00:00      1612.0    LM563          NaN   
7        4295909940  2016-11-13T00:00:00       843.0      NaN          NaN   
8        4350992505  2019-03-19T00:00:00      2154.0      NaN          NaN   
9        4272661584  2015-12-18T00:00:00      1822.0      NaN          NaN   
10       4270913382  2015-11-09T00:00:00       315.0      NaN          NaN   
11       4283394695  2016-05-19T00:00:00      1822.0       26   

# Data Analysis & Results

Include cells that describe the steps in your data analysis.

In [5]:
## YOUR CODE HERE
## FEEL FREE TO ADD MULTIPLE CELLS PER SECTION

# Ethics & Privacy

*Fill in your ethics & privacy discussion here*

# Conclusion & Discussion

*Fill in your discussion information here*